In [3]:
import gzip
import ast
import json
import pandas as pd
from datetime import datetime

In [4]:
def crearDFJSON(ruta):
    data = []
    with gzip.open(ruta, 'rb') as f:
        for line in f:
            data.append(json.loads(line))
    f.close
    return pd.DataFrame(data)

def crearDFAST(ruta):
    data = []
    with gzip.open(ruta, 'rb') as f:
        for line in f:
            data.append(ast.literal_eval(line.decode('utf-8')))
    f.close
    return pd.DataFrame(data)

def desanidarDF(df, columna):
    df_desanidado = df.explode(columna)
    return df_desanidado

def desanidar(df, columna):
    dfDesanidado = pd.json_normalize(df[columna])
    df = df.reset_index(drop=True)
    df = pd.concat([df, dfDesanidado], axis=1)
    df = df.drop(columna, axis=1)
    return df

In [5]:
steamGamesDF = crearDFJSON('../data/steam_games.json.gz')
usersItems = crearDFAST('../data/users_items.json.gz')

In [6]:
steamGamesDF = steamGamesDF.dropna().reset_index(drop=True)

columnas_elimnar = ['publisher', 'app_name', 'title', 'url',
       'tags', 'reviews_url', 'specs', 'price', 'early_access',
       'developer']
steamGamesDF = desanidarDF(columna='genres', df=steamGamesDF)
steamGamesDF = steamGamesDF.drop(columnas_elimnar, axis=1).reset_index(drop=True)
steamGamesDF.columns = ['Generos', 'Fecha_Lanzamiento', 'IdItem']


In [7]:
steamGamesDF

,Generos,Fecha_Lanzamiento,IdItem
0,Action,2018-01-04,761140
1,Casual,2018-01-04,761140
2,Indie,2018-01-04,761140
3,Simulation,2018-01-04,761140
4,Strategy,2018-01-04,761140
...,...,...,...
55607,Indie,2018-01-04,610660
55608,Racing,2018-01-04,610660
55609,Simulation,2018-01-04,610660
55610,Casual,2017-09-02,658870


In [8]:
columnas_elimnar = ['steam_id', 'user_url', 'items_count']
usersItems = usersItems.drop(columnas_elimnar, axis=1).reset_index(drop=True)
usersItems = desanidarDF(usersItems, 'items')
usersItems = desanidar(usersItems, 'items')
usersItems = usersItems.dropna()
usersItems = usersItems.drop_duplicates()

columnas_elimnar = ['playtime_2weeks', 'item_name']
usersItems = usersItems.drop(columnas_elimnar, axis=1).reset_index(drop=True)

usersItems.columns = ['IdUsuario','IdItem', 'Tiempo_Jugado']


In [9]:
steamGamesDF

,Generos,Fecha_Lanzamiento,IdItem
0,Action,2018-01-04,761140
1,Casual,2018-01-04,761140
2,Indie,2018-01-04,761140
3,Simulation,2018-01-04,761140
4,Strategy,2018-01-04,761140
...,...,...,...
55607,Indie,2018-01-04,610660
55608,Racing,2018-01-04,610660
55609,Simulation,2018-01-04,610660
55610,Casual,2017-09-02,658870


In [10]:
usersItems

,IdUsuario,IdItem,Tiempo_Jugado
0,76561197970982479,10,6.0
1,76561197970982479,20,0.0
2,76561197970982479,30,7.0
3,76561197970982479,40,0.0
4,76561197970982479,50,0.0
...,...,...,...
5094100,76561198329548331,346330,0.0
5094101,76561198329548331,373330,0.0
5094102,76561198329548331,388490,3.0
5094103,76561198329548331,521570,4.0


In [11]:
merged_df = pd.merge(usersItems, steamGamesDF, on='IdItem', how='inner')

In [12]:
columnas_elimnar = ['IdItem']
merged_df = merged_df.drop(columnas_elimnar, axis=1).reset_index(drop=True)

In [13]:
merged_df=merged_df.drop_duplicates().reset_index(drop=True)

In [14]:
df_reducido = merged_df[merged_df['Tiempo_Jugado'] > 0]

In [15]:
df_reducido.reset_index(drop=True)

,IdUsuario,Tiempo_Jugado,Generos,Fecha_Lanzamiento
0,76561197970982479,6.0,Action,2000-11-01
1,76561197970982479,7.0,Action,2003-05-01
2,76561197970982479,4733.0,Action,2010-07-12
3,76561197970982479,1853.0,Action,2004-11-01
4,76561197970982479,333.0,Action,2005-04-19
...,...,...,...,...
6609946,76561198329548331,3.0,Adventure,2015-09-01
6609947,76561198329548331,3.0,Free to Play,2015-09-01
6609948,76561198329548331,4.0,Casual,2016-08-24
6609949,76561198329548331,4.0,Free to Play,2016-08-24


In [16]:
df_reducido['Año_Lanzamiento'] = pd.to_datetime(df_reducido['Fecha_Lanzamiento'], errors='coerce').dt.year

/tmp/ipykernel_11002/3010855692.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_reducido['Año_Lanzamiento'] = pd.to_datetime(df_reducido['Fecha_Lanzamiento'], errors='coerce').dt.year


In [17]:
df_reducido = df_reducido.dropna(subset=['Año_Lanzamiento'])

In [18]:
df_reducido=df_reducido.drop(['Fecha_Lanzamiento'], axis=1).reset_index(drop=True)

In [19]:
# Agrupar por 'IdUsuario', 'Generos' y 'Año_Lanzamiento', sumando el tiempo jugado
df_agrupado = df_reducido.groupby(['IdUsuario', 'Generos', 'Año_Lanzamiento'])['Tiempo_Jugado'].sum().reset_index()


In [20]:
df_agrupado

,IdUsuario,Generos,Año_Lanzamiento,Tiempo_Jugado
0,--000--,Action,2009.0,5329.0
1,--000--,Action,2010.0,22.0
2,--000--,Action,2011.0,6522.0
3,--000--,Action,2012.0,109346.0
4,--000--,Action,2013.0,363.0
...,...,...,...,...
2770625,zzzmidmiss,Simulation,2014.0,16.0
2770626,zzzmidmiss,Sports,2014.0,16.0
2770627,zzzmidmiss,Strategy,2010.0,10.0
2770628,zzzmidmiss,Strategy,2011.0,69.0


In [21]:
# Dividir el DataFrame en dos partes
parte1 = df_agrupado.iloc[:len(df_agrupado)//2]
parte2 = df_agrupado.iloc[len(df_agrupado)//2:]

# Guardar cada parte en un archivo CSV separado
parte1.to_parquet('./UserForGenreParquet.parquet', index=False)

In [22]:
def UserForGenre(genero: str, df):
    df_genero = df[df['Generos'] == genero].copy()
    
    # Obtenemos el usuario con más horas jugadas para el género
    usuario_max_horas = df_genero.groupby('IdUsuario')['Tiempo_Jugado'].sum().idxmax()
    
    # Filtramos el DataFrame para obtener solo las filas del usuario con más horas
    df_usuario_max_horas = df_genero[df_genero['IdUsuario'] == usuario_max_horas]
    
    # No necesitamos convertir la columna 'Fecha_Lanzamiento' a tipo datetime
    
    # Eliminamos filas con valores NaN en la columna 'Año_Lanzamiento'
    df_usuario_max_horas = df_usuario_max_horas.dropna(subset=['Año_Lanzamiento'])

    # Agrupamos por 'Año_Lanzamiento' y sumamos las horas jugadas
    horas_por_año = df_usuario_max_horas.groupby('Año_Lanzamiento')['Tiempo_Jugado'].sum()
    
    # Creamos la lista de horas jugadas por año
    lista_horas_por_año = [{"Año": int(año), "Horas": horas} for año, horas in horas_por_año.items()]
    
    return {"Usuario con más horas jugadas para Género " + genero: usuario_max_horas, "Horas jugadas": lista_horas_por_año}


In [23]:
genero_dado = 'Indie' 
resultado = UserForGenre(genero_dado, df_agrupado)
print(resultado)

{'Usuario con más horas jugadas para Género Indie': 'REBAS_AS_F-T', 'Horas jugadas': [{'Año': 2001, 'Horas': 11.0}, {'Año': 2003, 'Horas': 1863.0}, {'Año': 2006, 'Horas': 1673.0}, {'Año': 2007, 'Horas': 1070.0}, {'Año': 2008, 'Horas': 1366.0}, {'Año': 2009, 'Horas': 28993.0}, {'Año': 2010, 'Horas': 20193.0}, {'Año': 2011, 'Horas': 95971.0}, {'Año': 2012, 'Horas': 148459.0}, {'Año': 2013, 'Horas': 168104.0}, {'Año': 2014, 'Horas': 326474.0}, {'Año': 2015, 'Horas': 750989.0}, {'Año': 2016, 'Horas': 806649.0}, {'Año': 2017, 'Horas': 33887.0}]}
